In [1]:
%pylab inline
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
import xgboost as xgb

Populating the interactive namespace from numpy and matplotlib


In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

In [3]:
categorical = ['Product_Info_1', 'Product_Info_2', 'Product_Info_3', 'Product_Info_5', 'Product_Info_6', 
               'Product_Info_7', 'Employment_Info_2', 'Employment_Info_3', 'Employment_Info_5', 'InsuredInfo_1',
               'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 'InsuredInfo_5', 'InsuredInfo_6', 
               'InsuredInfo_7', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 
               'Insurance_History_4', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 
               'Family_Hist_1', 'Medical_History_2', 'Medical_History_3', 'Medical_History_4',
               'Medical_History_5', 'Medical_History_6', 'Medical_History_7', 'Medical_History_8', 
               'Medical_History_9', 'Medical_History_10', 'Medical_History_11', 'Medical_History_12', 
               'Medical_History_13', 'Medical_History_14', 'Medical_History_16', 'Medical_History_17',
               'Medical_History_18', 'Medical_History_19', 'Medical_History_20', 'Medical_History_21', 
               'Medical_History_22', 'Medical_History_23', 'Medical_History_25', 'Medical_History_26',
               'Medical_History_27', 'Medical_History_28', 'Medical_History_29', 'Medical_History_30', 
               'Medical_History_31', 'Medical_History_33', 'Medical_History_34', 'Medical_History_35', 
               'Medical_History_36', 'Medical_History_37', 'Medical_History_38', 'Medical_History_39', 
               'Medical_History_40', 'Medical_History_41','Medical_History_1', 'Medical_History_15', 
               'Medical_History_24', 'Medical_History_32']

In [4]:
def one_hot_dataframe(data, cols, replace=False):
    """ Takes a dataframe and a list of columns that need to be encoded.
        Returns a 3-tuple comprising the data, the vectorized data,
        and the fitted vectorizor.
    """
    vec = DictVectorizer()
    mkdict = lambda row: dict((col, row[col]) for col in cols)
    vecData = pd.DataFrame(vec.fit_transform(data[cols].apply(mkdict, axis=1)).toarray())
    vecData.columns = vec.get_feature_names()
    vecData.index = data.index
    if replace is True:
        data = data.drop(cols, axis=1)
        data = data.join(vecData)
    return (data, vecData, vec)

In [5]:
train_ohd, _, _ = one_hot_dataframe(train, categorical,replace=True)
test_ohd, _, _ = one_hot_dataframe(test,categorical,replace=True)

In [6]:
features = train_ohd.columns.tolist()
features.remove("Id")
features.remove("Response")
train_features = train_ohd[features]
test_features = test_ohd[features]
y_train = train["Response"].values

In [11]:
print features

['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI', 'Employment_Info_1', 'Employment_Info_4', 'Employment_Info_6', 'Insurance_History_5', 'Family_Hist_2', 'Family_Hist_3', 'Family_Hist_4', 'Family_Hist_5', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'Medical_Keywo

In [14]:
param = {'max_depth':21, 'eta':0.025, 'min_child_weight':3, 'subsample' : 0.72 ,"early_stopping_rounds":10,
          "objective"   : "reg:linear",'eval_metric': 'rmse','colsample_bytree':0.68}
num_round = 1000

dtrain = xgb.DMatrix(train_features, label=y_train, missing=np.NaN)
dtest = xgb.DMatrix(test_features, missing=np.NaN)

watchlist  = [(dtrain,'train')]

bst = xgb.train(param, dtrain, num_round, watchlist)

y_test_bst=bst.predict(dtest)

[0]	train-rmse:5.572796
[1]	train-rmse:5.450461
[2]	train-rmse:5.330649
[3]	train-rmse:5.215033
[4]	train-rmse:5.107128
[5]	train-rmse:5.003222
[6]	train-rmse:4.902827
[7]	train-rmse:4.804594
[8]	train-rmse:4.701533
[9]	train-rmse:4.602847
[10]	train-rmse:4.505737
[11]	train-rmse:4.415092
[12]	train-rmse:4.330105
[13]	train-rmse:4.247566
[14]	train-rmse:4.167899
[15]	train-rmse:4.089145
[16]	train-rmse:4.014220
[17]	train-rmse:3.941883
[18]	train-rmse:3.863867
[19]	train-rmse:3.786620
[20]	train-rmse:3.720372
[21]	train-rmse:3.656265
[22]	train-rmse:3.586334
[23]	train-rmse:3.525853
[24]	train-rmse:3.458062
[25]	train-rmse:3.401159
[26]	train-rmse:3.336270
[27]	train-rmse:3.285084
[28]	train-rmse:3.223753
[29]	train-rmse:3.164523
[30]	train-rmse:3.117869
[31]	train-rmse:3.061466
[32]	train-rmse:3.018148
[33]	train-rmse:2.976220
[34]	train-rmse:2.924834
[35]	train-rmse:2.885638
[36]	train-rmse:2.847732
[37]	train-rmse:2.799971
[38]	train-rmse:2.764697
[39]	train-rmse:2.730742
[40]	train

In [13]:
print bst.get_fscore.im_self

In [15]:
def output_function(x):
    if x<1:
        return 1
    elif x>8:
        return 8
    else:
        return int(round(x))
        
y_test_bst_result=[output_function(y) for y in y_test_bst]

#write results

ids=test.Id.values.tolist()
n_ids=len(ids)
import csv

prediction_file = open("pythonxgb.csv", "w")
prediction_file_object = csv.writer(prediction_file)
prediction_file_object.writerow(["Id","Response"])
for i in range(0,n_ids):
    prediction_file_object.writerow([ids[i],y_test_bst_result[i]])